## Library & Settings

In [1]:
import cv2
import torch
import numpy as np
import csv
import pandas as pd

In [2]:
model=torch.hub.load('ultralytics/yolov5','custom',path='C:/Users/PC/Desktop/job/Object detect/result/last.pt',force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\PC/.cache\torch\hub\master.zip
YOLOv5  2023-10-10 Python-3.11.4 torch-2.1.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


## Function

Function here is to set up to class and help prediction process

In [12]:
def class_to_label(x):
    classes=model.names
    return classes[int(x)]

def score_frame(frame):
    frame=[frame]
    result=model(frame)
    labels,cord = result.xyxyn[0][:,-1],result.xyxyn[0][:,:-1]
    
    return labels, cord

def plot_boxes(results,frame):
    labels,cord =results
    n=len(labels)
    x_shape,y_shape = frame.shape[1],frame.shape[0]
    for i in range (n):
        row = cord[i]
        if row[4]>=0.4:
            x1,y1,x2,y2 = int (row[0]*x_shape),int(row[1]*y_shape) , int (row[2]*x_shape), int(row[3]*y_shape)
            bgr=(0,255,0)
            cv2.rectangle(frame,(x1,y1),(x2,y2),bgr,2)
            
            cv2.putText(frame,class_to_label(labels[i]),(x1,y1),cv2.FONT_HERSHEY_SIMPLEX,0.9,(0,255,0),2)
            
    return frame    

## Prediction

Make Prediction and labels the target,nametag 

In [13]:
video_path="C:/Users/PC/Desktop/job/Object detect/sample.mp4"
capture=cv2.VideoCapture(video_path)
frame_get=0
while True:
    success, img = capture.read()
    if not success:
        print("Frame Not Found")
        break
    results=score_frame(img)

    
    frame=plot_boxes(results,img)
    
    frame_get+=1
    cv2.imshow('Result',img)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

    
    
capture.release()
cv2.destroyWindow('Result')    

# Coordinate

In [3]:
fields=['x-1','y-1','x-2','y-2','confidence score','label','Name']
data=[]
frame_num=[]
def coordinate(frame,frame_get):
    result=model(frame)        
    for imgs in result.pandas().xyxy:
        for b in imgs.to_numpy():
            data.append(b)
            frame_num.append(frame_get)
            
            
    with open("C:/Users/PC/Desktop/job/Object detect/result/coordinate.csv", "w") as file:
        writer = csv.writer(file,skipinitialspace=True)
        writer.writerow(fields)
        writer.writerows(data)
        file.close()
    
    
    csv_input = pd.read_csv('C:/Users/PC/Desktop/job/Object detect/result/coordinate.csv')
    csv_input['Frame'] = frame_num
    csv_input.to_csv('output.csv', index=False)

In [4]:
video_path="C:/Users/PC/Desktop/job/Object detect/sample.mp4"
capture=cv2.VideoCapture(video_path)
frame_get=0
while True:
    success, img = capture.read()
    if not success:
        print("Frame Not Found")
        break
    result=coordinate(img,frame_get)
    frame_get+=1
    
    cv2.imshow('Result',img)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyWindow('Result')    

Frame Not Found
